# Department vs Company Salary

Your work as a data analyst for a FAANG company that tracks employee salaries over time. The company wants to understand how the average salary in each department compares to the company's overall average salary each month.

Write a query to compare the average salary of employees in each department to the company's average salary for March 2024. Return the comparison result as 'higher', 'lower', or 'same' for each department. Display the department ID, payment month (in MM-YYYY format), & the comparison result.

# Answer

I'll be reusing the `employee2` table:

```
SELECT * FROM employee2;
```

<img src = "employee2 Table.png" width = "600" style = "margin:auto"/>

But I will add a new table `salary`:

```
CREATE TABLE salary (
	salary_id smallint,
	employee_id smallint,
	amount smallint,
	payment_date timestamp
);

COPY salary
FROM '/YourDirectory/salary.csv'
WITH (FORMAT CSV, HEADER);

SELECT * FROM salary;
```

<img src = "salary Table.png" width = "600" style = "margin:auto"/>

Multiple tables means join.

```
WITH march_salary
AS (
	SELECT *
	FROM employee2
	LEFT JOIN salary USING (employee_id)
	WHERE to_char(payment_date, 'MM-YYYY') = '03-2024'
)
SELECT *,
	   (CASE WHEN dep_avg > comp_avg THEN 'higher'
	   	     WHEN dep_avg < comp_avg THEN 'lower'
	    ELSE 'same' END) AS comparison
FROM (
	SELECT department_id,
		   round(avg(salary), 2) AS dep_avg,
		   round((SELECT avg(salary) FROM march_salary), 2)
		   	   AS comp_avg
	FROM march_salary
	GROUP BY department_id
	ORDER BY department_id
);
```

<img src = "Department Average vs Company Average.png" width = "600" style = "margin:auto"/>

Department 3's average salary is higher than the company average. Department 1 & 2's average salaries are lower than the company average, though department 2's is very close.